# Kopp et al 2021 Training 

**Authorship:**
Adam Klie, *08/07/2022*
***
**Description:**
This notebook is a use case in EUGENe paper. It is used to showcase Janggu integrated functionality to classify junD binding sites.
***

In [2]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload 
%autoreload 2

import os
import logging
import torch
import numpy as np
import pandas as pd
import eugene as eu

Global seed set to 13
2022-08-25 15:39:42.248755: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-25 15:39:42.248808: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/workspaces/EUGENe/eugene/external/kipoi_veff/seqplotting_deps.py:36: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  min_coords = np.vstack(data.min(0) for data in polygons_data).min(0)
/workspaces/EUGENe/eugene/external/kipoi_veff/seqplotting_deps.py:37: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of N

In [3]:
# Configure EUGENe 
eu.settings.dataset_dir = "../../../_data/datasets/kopp21"
eu.settings.output_dir = "../../../_output/kopp21"
eu.settings.logging_dir = "../../../_logs/kopp21"
eu.settings.config_dir = "../../../_configs/kopp21"
eu.settings.batch_size = 128
eu.settings.dl_num_workers = 4
eu.settings.verbosity = logging.ERROR

# Load in the `SeqData`

In [4]:
sdata_sub = eu.dl.read_h5sd(
    filename="../../../_data/jund_processed_sub.h5sd",
)
sdata_sub.make_names_unique()
sdata_sub

SeqData object with = 2000 seqs
seqs = (2000,)
names = (2000,)
rev_seqs = (2000,)
ohe_seqs = (2000, 500, 4)
ohe_rev_seqs = (2000, 500, 4)
seqs_annot: 'chr', 'end', 'seq_len', 'start', 'target'
pos_annot: None
seqsm: None
uns: None

In [5]:
# TODO: in ETL!!! Split into train and test sets
eu.pp.train_test_split_data(
    sdata_sub, 
    train_key="train_val",
    chr = ["chr3"]
)
sdata_sub = eu.dl.concat([sdata_sub[sdata_sub["train_val"]][:90], sdata_sub[~sdata_sub["train_val"]][:10]], keys=["train", "val"])
sdata_sub["train_val"].value_counts()

SeqData object modified:
    seqs_annot:
        + train_val


# Model initialization 

In [11]:
from pytorch_lightning import seed_everything
def prep_new_model(
    seed,
    arch,
    config
):
    # Instantiate the model
    model = eu.models.load_config(
        arch=arch,
        model_config=config
    )

    # Set a seed
    seed_everything(seed)
    
    # Initialize the model prior to conv filter initialization
    eu.models.base.init_weights(model)

    # Return the model
    return model 

In [12]:
# Just make sure the model is taking in the proper data
model_types = ["FCN", "CNN", "RNN", "Hybrid", "Kopp21CNN"]
model_names = ["dsFCN", "dsCNN", "dsRNN", "dsHybrid", "Kopp21CNN"]
for model_name, model_type in zip(model_names, model_types):
    print(model_name, model_type)
    model = prep_new_model(0, model_type, os.path.join(eu.settings.config_dir, f"{model_name}.yaml"))
    if model_type == "RNN":
       sdataloader_sub = sdata_sub.to_dataset(transform_kwargs={"transpose": False}).to_dataloader() 
    else:
        sdataloader_sub = sdata_sub.to_dataset(transform_kwargs={"transpose": True}).to_dataloader()
    test_seqs = next(iter(sdataloader_sub))
    print(model(test_seqs[1], test_seqs[2]).size())
    print()

dsFCN FCN


/home/vscode/.local/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
Global seed set to 0
Global seed set to 0


No transforms given, assuming just need to tensorize).
torch.Size([100, 1])

dsCNN CNN
No transforms given, assuming just need to tensorize).


Global seed set to 0


torch.Size([100, 1])

dsRNN RNN
No transforms given, assuming just need to tensorize).


Global seed set to 0


torch.Size([100, 1])

dsHybrid Hybrid
No transforms given, assuming just need to tensorize).


Global seed set to 0


torch.Size([100, 1])

Kopp21CNN Kopp21CNN
No transforms given, assuming just need to tensorize).
torch.Size([100, 1])



In [13]:
# Train 5 models with 5 different random initializations
model_types = ["FCN", "CNN", "RNN", "Hybrid", "Kopp21CNN"]
model_names = ["dsFCN", "dsCNN", "dsRNN", "dsHybrid", "Kopp21CNN"]
#model_types = ["RNN"]
#model_names = ["dsRNN"]
trials = 5
for model_name, model_type in zip(model_names, model_types):
    for trial in range(1, trials+1):
        print(f"{model_name} trial {trial}")

        # Initialize the model
        model = prep_new_model(
            arch=model_type, 
            config=os.path.join(eu.settings.config_dir, f"{model_name}.yaml"),
            seed=trial
        )

        if model_type == "RNN":
            t_kwargs = transform_kwargs={"transpose": False}
        else:
            t_kwargs = transform_kwargs={"transpose": True}

        # Train the model
        eu.train.fit(
            model=model, 
            sdata=sdata_sub, 
            #gpus=1, 
            target="target",
            train_key="train_val",
            epochs=1,
            early_stopping_metric="val_loss",
            early_stopping_patience=5,
            transform_kwargs=t_kwargs,
            batch_size=64,
            num_workers=0,
            name=model_name,
            seed=trial,
            version=f"test_trial_{trial}",
            verbosity=logging.ERROR
        )
        # Get predictions on the training data
        eu.settings.dl_num_workers = 0
        eu.predict.train_val_predictions(
            model,
            sdata=sdata_sub, 
            target="target",
            train_key="train_val",
            transform_kwargs=t_kwargs,
            name=model_name,
            version=f"test_trial_{trial}",
            prefix=f"{model_name}_test_trial_{trial}_"
        )
        del model 
sdata_sub.write_h5sd(os.path.join(eu.settings.output_dir, "train_sub_predictions.h5sd"))

Global seed set to 1
Global seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type                      | Params
--------------------------------------------------------
0 | hp_metric | AUROC                     | 0     
1 | fcn       | BasicFullyConnectedModule | 1.1 M 
--------------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.232     Total estimated model params size (MB)


dsFCN trial 1
No transforms given, assuming just need to tensorize).
No transforms given, assuming just need to tensorize).


/home/vscode/.local/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /workspaces/EUGENe/tests/_logs/kopp21/dsFCN/test_trial_1/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

/home/vscode/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/home/vscode/.local/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
Global seed set to 1
/home/vscode/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of 

...

Metric val_loss improved. New best score: 0.555
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


No transforms given, assuming just need to tensorize).
No transforms given, assuming just need to tensorize).


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

SeqData object modified:
    seqs_annot:
        + Kopp21CNN_test_trial_5_target_predictions


---

## Scratch